# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [1]:
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from dotenv import load_dotenv
import warnings
import sys
import seaborn as sns
sys.path.append(r'notebooks\python_scripts')
from functions_variables import *
from data_importing_cleaning import *
from sub_df_creation import *

load_dotenv()
warnings.simplefilter(action='ignore', category=FutureWarning)

## Data Importing

In [2]:
# # Import and pickle raw df
# df = JSON_import(r'data/')
# df.to_pickle('data/!raw_df.pkl')

# # Cleaning missing data
# df= pd.read_pickle('data/!raw_df.pkl')
# df = missing_data(df)
# df.to_pickle('data/!cleaned_df.pkl')

df= pd.read_pickle('data/!cleaned_df.pkl')

df.head()


,property_id,baseball,basement,basketball,basketball_court,beach,beautiful_backyard,big_bathroom,big_lot,big_yard,...,beds,type,zip_code,state,coords_lon,coords_lat,city,address,street_view_url,county
0,9074430767,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3.000000,single_family,99801,Alaska,-134.593720,58.363950,Juneau,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,Juneau
1,9424983842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.207043,single_family,99801,Alaska,-92.285176,38.996694,Juneau,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,Juneau
3,9479068516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.207043,single_family,99801,Alaska,-92.285176,38.996694,Juneau,4515 Glacier Hwy,https://maps.googleapis.com/maps/api/streetvie...,Juneau
6,9879331943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.207043,single_family,99801,Alaska,-92.285176,38.996694,Juneau,17850 Point Stephens Rd,https://maps.googleapis.com/maps/api/streetvie...,Juneau
10,9521639574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.207043,single_family,99801,Alaska,-92.285176,38.996694,Juneau,9951 Stephen Richards Memorial Dr,https://maps.googleapis.com/maps/api/streetvie...,Juneau


In [4]:
tags_df = create_tags_df(df)
create_tags_totals_df(df)

,1.0
property_id,
9074430767,16.0
7412309719,11.0
7660490777,17.0
8846541030,15.0
7727981021,4.0
...,...
4542127284,8.0
3895826397,2.0
4941005485,2.0


In [ ]:
tags_df = create_tags_df(df)
tags_df=tags_df.T
tags_df.columns = tags_df.iloc[0]
tags_df = tags_df[1:]
tags_df = tags_df.apply(pd.value_counts)
tags_df=tags_df.T
tags_df = tags_df[[1.0]]
tags_df= tags_df.dropna(how='all')


,1.0
property_id,
9074430767,16.0
7412309719,11.0
7660490777,17.0
8846541030,15.0
7727981021,4.0
...,...
4542127284,8.0
3895826397,2.0
4941005485,2.0


## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

In [ ]:
import numpy as np
#Cleaning the type column
df['type']=df['type'].replace('condos', 'condo')
# apartments are rentals, condos are owned
df['type']=df['type'].replace('apartment', 'condo')
# 1 result and I looked it up
df['type']=df['type'].replace('condo_townhome_rowhome_coop', 'condo')
#1 result and I looked it up
df['type']=df['type'].replace('triplex_duplex', 'single_family')
# 3 results and I looked them all up
df['type']=df['type'].replace('other', 'single_family')
# removing year, baths, sqft, garage, stories, beds from land sales
df.loc[df['type']=='land', ['beds', 'baths','baths_3qtr', 'baths_full', 'baths_half', 'sqft', 'garage', 'stories']] = 0
# reverting year_built for land sales back to NaN
df.loc[df['type']=='land', 'year_built'] = np.nan

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [ ]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized